In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer # New 1
from sklearn.pipeline import Pipeline # New 2
from sklearn.feature_selection import SelectKBest, chi2 # New 3
from sklearn.pipeline import make_pipeline # New 4

In [6]:
df = pd.read_csv('/content/train.csv')

In [7]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Let's Plan

In [ ]:
# we have missing data in 2 column Age and embarked so 1st step will be 1. impute missing values
# In it's output we will send in 2nd column transformer which will do OHE 2 columns Sex and Embarked
# In it's ouput we will do scaling
# After this we will do feature selection and will take 5 columns only
# then will train the model using decision tree

In [8]:
df.drop(columns = ['PassengerId','Name','Ticket','Cabin'],inplace = True)

In [9]:
# Step-1 Train/test/split
X_train,X_test,Y_train,Y_test = train_test_split(df.drop(columns = ['Survived']),
                                                 df['Survived'],
                                                 test_size = 0.2,
                                                 random_state = 42)

In [10]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [11]:
Y_train.sample(5)

,Survived
83,0
228,0
729,0
763,1
428,0


In [12]:
#Imputation transformer
trf1 = ColumnTransformer([('imput_age',SimpleImputer(),[2]),
                          ('imput_embarked',SimpleImputer(strategy='most_frequent'),[6])
                          ],remainder = 'passthrough')

In [13]:
# One Hot Encoding
trf2 = ColumnTransformer([('ohe_sex_embarked',OneHotEncoder(sparse_output = False,handle_unknown = 'ignore'),[1,6])
                          ],remainder = 'passthrough')

In [14]:
# Scaling
trf3 = ColumnTransformer([('scale',MinMaxScaler(),slice(0,10))
                          ])

In [15]:
# Feature Selection
trf4 = SelectKBest(score_func = chi2,k = 8)

In [16]:
# Train the Model
trf5 = DecisionTreeClassifier()

# Create Pipeline

In [59]:
pipe = Pipeline([
                  ('trf1',trf1),
                  ('trf2',trf2),
                  ('trf3',trf3),
                  ('trf4',trf4),
                  ('trf5',trf5)
                  ])

# Pipeline Vs make_pipeline
pipeline requires naming of steps, make_pipelinedoes not.
(Same applies to column_transformer and make_column_tranformer)

In [58]:
# Alternate Syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [77]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('imput_age', SimpleImputer(), [2]),
                                 ('imput_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x795f6f0d44a0>),
 'trf5': DecisionTreeClassifier()}

In [60]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imput_age', SimpleImputer(),
                                                  [2]),
                                                 ('imput_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x795f6f0d44a0>)),
                ('trf5', DecisionTreeClassifier())])

# Explore The pipeline

In [61]:
# Display Pipeline
from sklearn import set_config
set_config(display = 'diagram')

In [62]:
# Train
pipe.fit(X_train,Y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imput_age', SimpleImputer(),
                                                  [2]),
                                                 ('imput_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x795f6f0d44a0>)),
                ('trf5', DecisionTreeClassifier())])

In [64]:
pipe.named_steps['trf1']

[('imput_age', SimpleImputer(), [2]),
 ('imput_embarked', SimpleImputer(strategy='most_frequent'), [6])]

In [65]:
pipe.named_steps['trf1'].transformers_

[('imput_age', SimpleImputer(), [2]),
 ('imput_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [68]:
# if we want to know that pahle wale mean value kya tha to list me se phla element nikal lunga
pipe.named_steps['trf1'].transformers_[0]

('imput_age', SimpleImputer(), [2])

In [69]:
# To get the simpleimputer which is a tuple
pipe.named_steps['trf1'].transformers_[0][1]

SimpleImputer()

In [71]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_
# output will be the mean value

array([29.49884615])

In [74]:
# now if we want to see 2nd wale me most frequent vlaue kya hai then we will write 1
pipe.named_steps['trf1'].transformers_[1]

('imput_embarked', SimpleImputer(strategy='most_frequent'), [6])

In [75]:
pipe.named_steps['trf1'].transformers_[1][1]

SimpleImputer(strategy='most_frequent')

In [76]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [81]:
# Predict
Y_pred = pipe.predict(X_test)

In [82]:
Y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [83]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.6256983240223464

# Cross Validation using pipelines

In [84]:
#Cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,Y_train,cv = 5,scoring = 'accuracy').mean()

0.6391214419383433

# Grid Search using pipeline

In [85]:
#gridsearchcv
params = {'trf5__max_depth':[1,2,3,4,5,None]}

In [86]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv = 5,scoring = 'accuracy')
grid.fit(X_train,Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('imput_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('imput_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x795f6f0d44a0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [87]:
grid.best_score_

0.6391214419383433

In [88]:
grid.best_params_

{'trf5__max_depth': 2}

# Exporting the pipeline

In [90]:
# Export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))